# TensorFlow Debugging Exercise

This notebook demonstrates common TensorFlow bugs and their fixes for MNIST digit classification.

## Part 1: Buggy Code (10 Common Errors)

The following code contains 10 intentional bugs. Can you spot them all?

In [ ]:
# BUGGY CODE - DO NOT RUN THIS CELL
# This is for reference only

"""
import tensorflow as tf
from tensorflow import keras

# Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# BUG 1: No normalization
# BUG 2: Wrong shape (not flattened)
# BUG 3: Labels not one-hot encoded

# BUG 4: Input shape mismatch
# BUG 5: Only 5 output neurons (should be 10)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    keras.layers.Dense(5, activation='softmax')  # Wrong!
])

# BUG 6: Wrong loss function
# BUG 7: Learning rate too high
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=10.0),
    loss='binary_crossentropy',  # Wrong!
    metrics=['accuracy']
)

# BUG 8: Batch size larger than dataset
model.fit(x_train, y_train, batch_size=100000, epochs=5)
"""

print("Buggy code shown above for reference")
print("See the fixes below!")

## Part 2: Fixed Code with Explanations

### Step 1: Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

print("="*80)
print("FIXED TENSORFLOW SCRIPT - MNIST CLASSIFIER")
print("="*80)
print(f"TensorFlow version: {tf.__version__}")

### Step 2: Load and Inspect Data

In [ ]:
# Load MNIST dataset
print("\n📥 Loading MNIST dataset...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(f"Original training data shape: {x_train.shape}")
print(f"Original test data shape: {x_test.shape}")
print(f"Original training labels shape: {y_train.shape}")
print(f"Label range: {y_train.min()} to {y_train.max()}")
print(f"Pixel value range: {x_train.min()} to {x_train.max()}")

### Step 3: FIX #1 - Normalize Data

In [ ]:
print("\n🔧 FIX 1: Normalizing data to [0, 1] range...")
print("   Original range: [0, 255]")

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

print(f"   Fixed range: [{x_train.min():.2f}, {x_train.max():.2f}]")
print("   ✅ Data normalized!")

### Step 4: FIX #2 - Reshape Data

In [ ]:
print("\n🔧 FIX 2: Reshaping data for Dense layers...")
print(f"   Original shape: {x_train.shape}")

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

print(f"   Fixed shape: {x_train.shape}")
print("   ✅ Data reshaped from (28, 28) to (784,)")

### Step 5: FIX #3 - One-Hot Encode Labels

In [ ]:
print("\n🔧 FIX 3: Converting labels to one-hot encoding...")
print(f"   Original labels shape: {y_train.shape}")
print(f"   Sample original labels: {y_train[:5]}")

y_train_categorical = keras.utils.to_categorical(y_train, 10)
y_test_categorical = keras.utils.to_categorical(y_test, 10)

print(f"   Fixed labels shape: {y_train_categorical.shape}")
print(f"   Sample one-hot encoded label:\n   {y_train_categorical[0]}")
print("   ✅ Labels one-hot encoded!")

### Step 6: FIX #4 & #5 - Build Correct Model Architecture

In [ ]:
print("\n🏗️  Building model...")

model = keras.Sequential([
    # FIX 4: Correct input shape (784,) matches reshaped data
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    
    # FIX 5: Correct number of output neurons (10 for MNIST digits 0-9)
    layers.Dense(10, activation='softmax')
])

print("\n🔧 FIX 4: Input shape now matches data shape (784,)")
print("🔧 FIX 5: Output layer has 10 neurons for 10 classes")

print("\n📋 Model Summary:")
model.summary()

### Step 7: FIX #6 & #7 - Correct Compilation

In [ ]:
print("\n🔧 FIX 6: Using categorical_crossentropy for multi-class classification")
print("🔧 FIX 7: Using appropriate learning rate (0.001)")

# FIX 7: Use appropriate learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Reasonable learning rate

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # FIX 6: Correct loss function
    metrics=['accuracy']
)

print("✅ Model compiled successfully!")

### Step 8: FIX #8 - Train with Appropriate Batch Size

In [ ]:
print("\n🔧 FIX 8: Using appropriate batch size (128)")
print(f"   Dataset size: {len(x_train)}")
print(f"   Batch size: 128")
print(f"   Batches per epoch: {len(x_train) // 128}")

print("\n🎯 Training model...")
history = model.fit(
    x_train, y_train_categorical,
    batch_size=128,  # FIX 8: Appropriate batch size
    epochs=5,
    validation_split=0.2,
    verbose=1
)

### Step 9: FIX #9 - Evaluate with Properly Preprocessed Data

In [ ]:
print("\n📊 Evaluating model...")
test_loss, test_acc = model.evaluate(x_test, y_test_categorical, verbose=0)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")
print("✅ FIX 9: Test data preprocessed correctly!")

### Step 10: FIX #10 - Correct Predictions

In [ ]:
print("\n🔮 Making predictions...")
predictions = model.predict(x_test[:5], verbose=0)
print(f"Predictions shape: {predictions.shape}")
print("✅ FIX 10: Predictions have correct shape (samples, 10)")

print(f"\nSample predictions (probabilities for each class):")
for i in range(5):
    predicted_class = np.argmax(predictions[i])
    actual_class = y_test[i]
    confidence = predictions[i][predicted_class]
    print(f"   Sample {i}: Predicted={predicted_class}, Actual={actual_class}, "
          f"Confidence={confidence:.2%}")

### Step 11: Visualize Training History

In [ ]:
import matplotlib.pyplot as plt

print("\n📈 Training History:")
print(f"   Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"   Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"   Final training loss: {history.history['loss'][-1]:.4f}")
print(f"   Final validation loss: {history.history['val_loss'][-1]:.4f}")

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True)

# Loss plot
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Summary of All Fixes

In [ ]:
print("\n" + "="*80)
print("✅ ALL FIXES APPLIED")
print("="*80)

fixes = [
    "✓ Data normalized to [0, 1]",
    "✓ Data reshaped to (samples, 784)",
    "✓ Labels converted to one-hot encoding",
    "✓ Input shape matches data shape",
    "✓ Output layer has 10 neurons",
    "✓ Using categorical_crossentropy loss",
    "✓ Using appropriate learning rate (0.001)",
    "✓ Using appropriate batch size (128)",
    "✓ Proper data preprocessing for evaluation",
    "✓ Predictions have correct shape"
]

for fix in fixes:
    print(f"   {fix}")

print("\n" + "="*80)
print("✅ SCRIPT COMPLETED SUCCESSFULLY (ALL BUGS FIXED)")
print("="*80)

## Debugging Checklist

Use this checklist for future TensorFlow projects:

### Before Training:
- [ ] Data normalized to [0, 1] or standardized?
- [ ] Data shape matches model input?
- [ ] Labels properly encoded (one-hot or integer)?
- [ ] Train/test split done correctly?

### Model Architecture:
- [ ] Input shape matches data shape?
- [ ] Output neurons = number of classes?
- [ ] Appropriate activation functions?
- [ ] Reasonable number of layers/neurons?

### Compilation:
- [ ] Correct loss function for task?
- [ ] Appropriate optimizer?
- [ ] Reasonable learning rate?
- [ ] Relevant metrics specified?

### Training:
- [ ] Batch size reasonable?
- [ ] Number of epochs appropriate?
- [ ] Validation split or validation data?

### Evaluation:
- [ ] Test data preprocessed same as training?
- [ ] Using correct evaluation metrics?
- [ ] Predictions interpreted correctly?